In [6]:
# %% 
import os, cv2
from tqdm import tqdm

# paths—you can swap in filtered_infected if you like
filtered_folder = '../data/filtered/healthy_eye'
test_out        = '../data/temp_crops/healthy_eye_haar'
os.makedirs(test_out, exist_ok=True)

for f in os.listdir(test_out):
    os.remove(os.path.join(test_out, f))

In [4]:


# load Haar cascade for eyes (comes with opencv-python)
eye_cascade = cv2.CascadeClassifier(
    cv2.data.haarcascades + 'haarcascade_eye.xml'
)
# %% Wipe just the contents
for f in os.listdir(test_out):
    os.remove(os.path.join(test_out, f))

for fname in tqdm(os.listdir(filtered_folder), desc="Haar Eye Split Test"):
    src_path = os.path.join(filtered_folder, fname)
    img_bgr  = cv2.imread(src_path)
    if img_bgr is None:
        continue

    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)
    # detect eyes (you can tweak scaleFactor/minNeighbors)
    detections = eye_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=6)
    if len(detections) >= 2:
        # sort by x-position, take leftmost two
        eyes = sorted(detections, key=lambda b: b[0])[:2]
        for i,(x,y,w_e,h_e) in enumerate(eyes):
            crop = img_bgr[y:y+h_e, x:x+w_e]
            out_name = f"{os.path.splitext(fname)[0]}_{['left','right'][i]}.jpg"
            cv2.imwrite(os.path.join(test_out, out_name), crop)
    elif len(detections) == 1:
        x,y,w_e,h_e = detections[0]
        crop = img_bgr[y:y+h_e, x:x+w_e]
        out_name = f"{os.path.splitext(fname)[0]}.jpg"
        cv2.imwrite(os.path.join(test_out, out_name), crop)
    else:
        # fallback: save the full image if no eyes found
        out_name = f"{os.path.splitext(fname)[0]}_full.jpg"
        cv2.imwrite(os.path.join(test_out, out_name), img_bgr)

print("Done! Crops written to", test_out)

Haar Eye Split Test: 100%|██████████| 788/788 [00:03<00:00, 219.71it/s]

Done! Crops written to ../data/temp_crops/healthy_eye_haar
